In [1]:
import math
import random
import numpy as np

In [2]:
RU_8p = {1 : {"transition" : "P",
               "reward": 2,
               "next state": "TU_10p"
              },
         2 : {"transition" : "R",
               "reward" : 0,
               "next state" : "RU_10p"
               },
         3 : {"transition" : "S",
                "reward" : -1,
                "next state": "RD_10p"
               }
        }

In [3]:
TU_10p = {1: {"transition" : "P",
               "reward": 2,
               "next state": "RU_10a"
             },
         2: {"transition" : "R",
               "reward": 0,
               "next state": "RU_8a"
            }
         }

RU_10p = {1 : {"transition" : "R",
               "reward": 0,
               "next state": "RU_8a"
              },
         2 : {"transition" : "P",
               "reward" : 0,
               "next state" : "intermediate1"
               },
         3 : {"transition" : "S",
                "reward" : -1,
                "next state": "RD_8a"
               }
        }

RD_10p = {1 : {"transition" : "R",
               "reward": 0,
               "next state": "RD_8a"
              },
         2 : {"transition" : "P",
               "reward" : 0,
               "next state" : "intermediate2"
               },
        }

In [4]:
intermediate1 = {1 : {"transition" : "Tran",
                       "reward": 2,
                       "next state": "RU_8a"
                      },
                 2 : {"transition" : "Tran",
                       "reward" : 2,
                       "next state" : "RU_10a"
                       },
                }

intermediate2 = {1 : {"transition" : "Tran",
                       "reward": 2,
                       "next state": "RD_8a"
                      },
                 2 : {"transition" : "Tran",
                       "reward" : 2,
                       "next state" : "RD_10a"
                       },
                }

In [5]:
RU_8a = {1 : {"transition" : "P",
               "reward": 2,
               "next state": "TU_10a"
              },
         2 : {"transition" : "R",
               "reward" : 0,
               "next state" : "RU_10a"
               },
         3 : {"transition" : "S",
                "reward" : -1,
                "next state": "RD_10a"
               }
        }

RD_8a = { 1 : {"transition" : "R",
               "reward" : 0,
               "next state" : "RD_10a"
               },
         2 : {"transition" : "P",
               "reward": 2,
               "next state": "TD_10a"
              }
        }

In [6]:
TU_10a = { 1 : {"transition" : "any",
               "reward" : -1,
               "next state" : "11aClass"
               }
        }

RU_10a = { 1 : {"transition" : "any",
               "reward" : 0,
               "next state" : "11aClass"
               }
        }

RD_10a = { 1 : {"transition" : "any",
               "reward" : 4,
               "next state" : "11aClass"
               }
        }

TD_10a = { 1 : {"transition" : "any",
               "reward" : 3,
               "next state" : "11aClass"
               }
         }

In [7]:
states = {"RU_8p" : RU_8p,
          "TU_10p" : TU_10p,
          "RU_10p" : RU_10p,
          "RD_10p" : RD_10p,
          "intermediate1" : intermediate1,
          "intermediate2" : intermediate2,
          "RU_8a" : RU_8a,
          "RD_8a" : RD_8a,
          "TU_10a" : TU_10a,
          "RU_10a" : RU_10a,
          "RD_10a" : RD_10a,
          "TD_10a" : TD_10a
         }

In [8]:
#------------------------------------ Policy Evaluation --------------------------------------------
policy= [1, 1, 2, 2, 0, 0, 1, 2, 1, 1, 1, 1]    #initial policy is numbered so that Party is preferred always acc to ques
Vs = np.zeros(len(states))
values = list(states.keys())

def policy_evaluation(policy, Vs):
    values = list(states.keys())
    delta = 1
    theta = 0.001
    while (delta >= theta):
        delta = 0
        for current_state in states:
            value1 = list(states[current_state].keys())
            current_index = values.index(current_state)
            action = policy[current_index]
            v = Vs[current_index]
            if(action == 0):
                reward_1 = states[current_state][1]["reward"]
                reward_2 = states[current_state][2]["reward"]
                next_state_1 = states[current_state][1]["next state"]
                next_state_2 = states[current_state][2]["next state"]
                next_index_1 = values.index(next_state_1)
                next_index_2 = values.index(next_state_2)
                Vs[current_index] = (reward_1 + Vs[next_index_1] + reward_2 + Vs[next_index_2])/2.0

            elif(current_state == 'TU_10a' or current_state == 'RU_10a' or current_state == 'RD_10a' or current_state == 'TD_10a'):
                current_reward = states[current_state][action]["reward"]
                Vs[current_index] = current_reward

            else:            
                next_state = states[current_state][action]["next state"]
                current_reward = states[current_state][action]["reward"]
                next_index = values.index(next_state)
                Vs[current_index] = current_reward + Vs[next_index]
            delta = max(delta, abs(v-Vs[current_index]))
        if (delta < theta):
            print('Value of states: ', Vs)
            policy_improvement()
            break
    

In [9]:
#------------------------------------------- Policy Improvement --------------------------------
def policy_improvement():
    for current in states:
        value1 = list(states[current].keys())
        current_index = values.index(current)
        old_action = policy[current_index]
        if(old_action != 0 and current != 'TU_10a' and current != 'RU_10a' and current != 'RD_10a' and current != 'TD_10a'):
            l = len(states[current])
            max_val = 0
            new_action = old_action
            for j in range(1, (l+1)):
                next_state = states[current][j]["next state"]
                current_reward = states[current][j]["reward"]
                next_index = values.index(next_state)
                new_val = current_reward + Vs[next_index]
                if(new_val > max_val):
                    max_val = new_val
                    new_action = j
            if (old_action != new_action):
                policy[current_index] = new_action
                print('Policy change in:',current, '   Old action = ', states[current][old_action]['transition'], '   New Action = ', states[current][new_action]['transition'])
                print('New Policy: ', policy,'\n')
                policy_evaluation(policy, Vs)
print('Initial Policy: ', policy,'\n')
policy_evaluation(policy, Vs)
print('Final Policy:', policy)
for state in states:
    if (state != 'intermediate1' and state != 'intermediate2'):
        current_index = values.index(state)
        action = policy[current_index]
        print(state, ' -->', states[state][action]['transition'] )
    

Initial Policy:  [1, 1, 2, 2, 0, 0, 1, 2, 1, 1, 1, 1] 

Value of states:  [ 4.   2.   2.5  6.5  2.5  6.5  1.   5.  -1.   0.   4.   3. ]
Policy change in: RU_8p    Old action =  P    New Action =  S
New Policy:  [3, 1, 2, 2, 0, 0, 1, 2, 1, 1, 1, 1] 

Value of states:  [ 5.5  2.   2.5  6.5  2.5  6.5  1.   5.  -1.   0.   4.   3. ]
Policy change in: RU_10p    Old action =  P    New Action =  S
New Policy:  [3, 1, 3, 2, 0, 0, 1, 2, 1, 1, 1, 1] 

Value of states:  [ 5.5  2.   4.   6.5  2.5  6.5  1.   5.  -1.   0.   4.   3. ]
Policy change in: RU_8a    Old action =  P    New Action =  S
New Policy:  [3, 1, 3, 2, 0, 0, 3, 2, 1, 1, 1, 1] 

Value of states:  [ 5.5  2.   4.   6.5  3.5  6.5  3.   5.  -1.   0.   4.   3. ]
Policy change in: TU_10p    Old action =  P    New Action =  R
New Policy:  [3, 2, 3, 2, 0, 0, 3, 2, 1, 1, 1, 1] 

Value of states:  [ 5.5  3.   4.   6.5  3.5  6.5  3.   5.  -1.   0.   4.   3. ]
Final Policy: [3, 2, 3, 2, 0, 0, 3, 2, 1, 1, 1, 1]
RU_8p  --> S
TU_10p  --> R
RU_10p  